# Imports

In [22]:
import random
from time import time
import matplotlib.pyplot as plt

# Classes

The following are three classes namly Node, Function, and Service. These classes are used to represent nodes, functions, and services. Each class variable is illustrated in the class’s constructor. Each class method is illustrated inside the method. There are some parameters mentioned in the descriptions (e.g., the ones enclosed in single quotes such as 'number_of_nodes'). These parameters will be illustrated in Parameter section (e.g., global variables) or in the creation of class instances (e.g., class parameters). 

In [23]:
class Node(object):
    def __init__(self, ID, supported_function_IDs,  total_storage):
        """
        'ID' is a unique number assigned to a node. It is from 1 to 'number_of_nodes'.
        'hosted_functions' is a list of function IDs that a given node can process.
        'total_storage' is the memory size of a node.
        'available_storage' is a float representing the available memory size of a node. Initially it is 'total_storage'.
        'mapped_functions' is a list of functions that are added to a node. Initially, it is empty.
        'queue_size' is the number of functions waiting in the queue. Initially, it is zero.
        'expected_completion_time' is the time of completing the last queued function. It is represented by 'PI' symbol 
        in the paper. Initially, it is zero.
        """
        self.ID = ID
        self.supported_function_IDs = supported_function_IDs
        self.total_storage = total_storage
        self.available_storage = total_storage
        self.mapped_functions = []
        self.queue_size = 0
        self.expected_completion_time = 0

    def has_space(self, function_demand):
        """
        This method checks if a given function could be added to the node. It calculates the 
            available storage (expected available storage) if the function is to be added. If the 
            expected available storage is negative, the method returns False. Otherwise, it returns True.
        
        Variables:
            'function_demand' represents how much buffer is needed by a function.
            'expected_available_storage' is the available_storage after adding a function to the node.
        """
        
        expected_available_storage = self.available_storage - function_demand
        
        if (expected_available_storage < 0):
            return False
        
        else:
            return True
        
    def add_function(self, function):
        """
        This method appends the last added function to the list 'mapped_functions'.
        
        Variables:
            'function' is an instance of the class Function. It represents the last added function.
        """
        self.mapped_functions.append(function)
        
    def update_available_storage(self, function_demand):
        """
        This method updates the class variable 'available_storage' if a function is added.
        
        Variables:
            'function_demand' represents how much buffer is needed by a function.
        """
        
        self.available_storage = self.available_storage - function_demand
        
    def update_expected_completion_time(self, CT_last_added_function):
        """
        This method updates the class variable 'expected_completion_time' if a function is added. 'expected_completion_time'
            is set to the competion time of the added function (e.g., the last added function) to the node's queue.
         
        Variables:
            'CT_last_added_function' is the completion time of the last added function.
        """

        self.expected_completion_time = CT_last_added_function

    def update_queue_size(self):
        """
        This method updates the class variable 'queue_size' if a function is added. The method increments 
        the queue size by 1.
        """
        
        self.queue_size = self.queue_size + 1
        
    def free_resources(self, function_demand):
        """
        This method frees resources held by a function when the function is served.
        
        Variables:
            'function_demand' represents how much buffer is needed by a function.
        """
        
        # Decrement 'queue_size'.
        self.queue_size = self.queue_size - 1
        
        # 'available_storage' by adding 'function_demand' to it.
        self.available_storage = self.available_storage + function_demand
        
        # If there are no functions in the queue, then set 'expected_completion_time' to zero.
        if(self.queue_size == 0):
            self.expected_completion_time = 0

In [24]:
class Function(object):
    def __init__(self, ID, buffer_demand):
        """
        'ID' is a unique number assigned to a function. It is from 1 to 'number_of_functions'.
        'buffer_demand' is a float representing how much memory is needed to store the function. It is 
            randomly generated, and it is from 7.5 to 10.
        'last_function' is a Boolean to indicate the last function of a service. If the function is 
            the last in a given service, then 'last_function' is True. Otherwise, it is False.
        'start_time' is the start time of a function. It is set in the 'Greedy_Function_Mapping' method.
        'completion_time' is the completion time of a function. It is set in the 'Greedy_Function_Mapping' method.
        'hosting_node' represents the node that hosts the function.
        """
        self.ID = ID
        self.buffer_demand = buffer_demand
        self.last_function = False
        self.start_time = 0
        self.completion_time = 0
        self.hosting_node = None

In [25]:
class Service(object):
    def __init__(self, ID, arrival_time, requested_functions, processing_deadline):
        """
        'ID' is a unique number assigned to a service. It is from 1 to the number of services to 
            be simulated (e.g., N).
        'arrival_time' is the arrival time of a service.
        'requested_functions' is a list holding the requested functions.
        'processing_deadline' a float representing the deadline of a service.
        """
        self.ID = ID
        self.arrival_time = arrival_time
        self.requested_functions = requested_functions
        self.processing_deadline = processing_deadline

# Mapping

The following is an implementation of the Greedy Best Availability (GBA) algorithm. The criterion is to sort candidate nodes (e.g., nodes that can process a given function) based on the least 'expected_completion_time'. For instance, a function is mapped to the node with the earliest 'expected_completion_time'. The method returns True if all functions of a service are mapped and scheduled. Otherwise, the method returns False indicating that a function could not be mapped and scheduled. So, the service is rejected.

In [26]:
def Greedy_Function_Mapping (service, nodes):
    
    """
    'service' is an arrived service with a set of functions.
    'nodes' is a global list of nodes. Each entry in the list is an instance of the class Node. The list
        is illustrated in Parameter section.
    """
    
    """
    'initial_values' and 'initial_IDs' lists are used to store initial values of nodes (e.g., values before the arrival 
        of the service). This is needed because the method will change some attributes of nodes (e.g., mapped functions and 
        completion times of functions) while it tries to map functions to nodes. So, we need to store the initial values so 
        that, in case a function is not mapped, we can restore the original values. For example, if a service arrives with 5 
        functions. The method will map the functions to nodes which will cause some changes in nodes. However, if the fourth 
        function is not mapped, then the service is rejected. So, there is a need to restore the initial state of the nodes 
        (e.g., the state before the arrival of the service).
    """
    initial_values = []
    initial_IDs = []
    
    # A Boolean variable to keep track of the first function of a service.
    first_function = True
    
    # Loop over functions of a service.
    for function in service.requested_functions:

        # Create a dummy list to hold candidate nodes for a function.
        candidate_nodes = []

        """
        Check if the current function is the first function in the service. This is needed to assign 'previous_time' 
            as the arrival time of the service.
            
        'previous_time' indicates the competion time of the previous function in a given service.
        """
        
        if (first_function):
            
            # Assign the previous time to arrival time of 'service'
            previous_time = service.arrival_time
            first_function = False

        # Loop over nodes to collect candidate nodes for the function
        for node in nodes:
            
            
            function.completion_time = processing_times[function.ID][node.ID] + max(node.expected_completion_time, previous_time)
            
            if ((B_binary[function.ID][node.ID] == 1) and (node.has_space(function.buffer_demand)) and 
                (function.completion_time <= service.processing_deadline)):
                
                candidate_nodes.append(node)
                
        # If no candidate nodes are found, reject service and restore initial state of the nodes.
        if(len(candidate_nodes) == 0):
            
            # Restore initial values of nodes
            for node in nodes:
                for IV in initial_values:
                    if node.ID == IV[0]:
                        node.queue_size = IV[1]
                        node.available_storage = IV[2]
                        node.expected_completion_time = IV[3]
            
            # Servicce rejected, return
            return False

        # Sort nodes on 'candidate_nodes' according to the least 'expected_completion_time' in assending order.
        # The node with the lowest 'expected_completion_time' will be the first after sorting.
        
        candidate_nodes.sort(key=lambda x: x.expected_completion_time)
        
        # Collect initial values of nodes
        if not candidate_nodes[0].ID in initial_IDs:
            initial_IDs.append(candidate_nodes[0])
            
            # Collect initial values of a node
            initial_values.append([candidate_nodes[0].ID, candidate_nodes[0].queue_size, 
                                   candidate_nodes[0].available_storage, candidate_nodes[0].expected_completion_time])

            
        # Map 'function' to best node according to the above sort.
        candidate_nodes[0].add_function(function)
        
        
        # Update 'hosting_node' of the function.
        function.hosting_node = candidate_nodes[0]
        
        # Set the start time of the function.
        function.start_time =  max(candidate_nodes[0].expected_completion_time, previous_time)

        # Update queue, available storage, and expected completion time of the node
        candidate_nodes[0].update_queue_size()
        candidate_nodes[0].update_available_storage(function.buffer_demand)
        candidate_nodes[0].update_expected_completion_time(function.completion_time)
        
        # Update previous time
        previous_time = function.completion_time
            
    return True

# Generate Nodes

This method generates nodes of the network. It is called every time a new run of the simulation is to start.

In [27]:
def generate_nodes():
    global nodes
    for i in range (number_of_nodes):

        # Randomly select the number of functions from 1 to 7.
        No_supported_function_IDs = random.randint(1, 7)

        # Randomly sample function IDs.
        supported_function_IDs = random.sample(function_IDs, No_supported_function_IDs)

        # Randomly select the storage size of a node from 75 to 100.
        storage = random.randint(75, 100)

        for functionID in supported_function_IDs:

            # Randomly set processing times in the matrix 'processing_times' from 15 to 30.
            processing_times[functionID][i+1] = random.uniform(15, 30)

        # Append a node object to 'nodes' list
        nodes.append(Node(i + 1, supported_function_IDs, storage))

    # Update the binary matrix
    for i in range (len(nodes)):
        for function_id in nodes[i].supported_function_IDs:
            B_binary[function_id][nodes[i].ID] = 1

    # Just to make sure that a function is at least supported by a node.
    # If the sum of a row is zero, then the corresponding function is not supported in any node.
    for row in range(1, number_of_functions + 1):

        row_sum = 0
        for column in range(1, number_of_nodes + 1):
            row_sum = row_sum + B_binary[row][column]

        if(row_sum == 0):
            print("Row " + str(row) + " is all zeros")

## Reset Global Variables

This method is used to reset global varibales that are modified by a simulation run. It is called every time a new run of the simulation is to start.

In [28]:
def reset_global():
    
    # Parameters:
    global clock, arrivals, departures, rejected, active_services, service_ID
    
    clock = 0.0
    
    # Counts of simulated arrivals, departures, rejected services, and services currently in the system.
    arrivals = 0
    departures = 0
    rejected = 0
    active_services = 0
    service_ID = 0
    
    # Lists and matrices:
    global nodes, evList, B_binary, processing_times
    
    nodes = []
    evList = []
    
    B_binary = [[0 for x in range(number_of_nodes + 1)] for y in range(number_of_functions + 1)] 
    processing_times = [[0 for x in range(number_of_nodes + 1)] for y in range(number_of_functions + 1)] 
    
    
    # Output lists:
    global Acceptance_Ratio, Time_Gaps, Flow_Time, Queue_Length, Total_Cost, Total_Revenue, arrival_times
    
    Acceptance_Ratio = []
    Time_Gaps = []
    Flow_Time = []
    Queue_Length = []
    Total_Cost = []
    Total_Revenue = []
    
    arrival_times = []

# Events and Event Handlers

The following is the implementation of events and event handlers.

In [29]:
def Gen_Arr_event(clock):
    
    arrival_time = clock + random.expovariate(1 / lamda)
    ev = (arrival_time, handle_Arr_event)
    insert(ev)
    
def handle_Arr_event(clock):
    global service_ID, arrivals, active_services
    
    # Clock = clock + inter arrival time of current service = arrival time of current service.
    # N is the number of services to be simulated
    if (arrivals < N):
        
        arrivals = arrivals + 1
        service_ID = service_ID + 1

        # A list to hold requested functions.
        requested_functions = []

        # Randomly get the number of requested functions from 5 to 10.
        No_requested_functions = random.randint(5,10)

        # Randomly sample function IDs from the list 'function_IDs'.
        sampled_function_IDs = random.sample(function_IDs, No_requested_functions)

        # Loop aver 'sampled_function_IDs', and generate functions of type 'Function'.
        for f in sampled_function_IDs:
            function_ID = f
            
            # Randomly get 'buffer_demand' from 7.5 to 10
            buffer_demand = random.uniform(7.5, 10) 
            
            requested_functions.append(Function(function_ID, buffer_demand))

        # Set 'last_function' of last function of 'requested_functions' to True.
        requested_functions[-1].last_function = True
        
        # Randomly get 'processing_deadline' from 5000, 10,000.
        processing_deadline = random.randint(5000, 10000)
        
        # Clock = arrival time of current service
        arrived_service = Service(service_ID, clock, requested_functions, clock + processing_deadline)
        
        
        if (Greedy_Function_Mapping(arrived_service, nodes)):
            
            active_services = active_services + 1
            
            # Schedule next 'Gen_manage_FE' event. Pass 'arrived_service' to the event.
            Gen_manage_FE(clock, arrived_service)
            
        else:
            # Schedule next 'Gen_reject_event' event
            Gen_reject_event(clock)
        
        
        # Schedule next arrival event.
        Gen_Arr_event(clock)

def Gen_manage_FE(clock, *args):
    """
    FE stands for Function Event. This event could be handled by 'handle_Arr_event'. However, we decided to have 
        a separate event for it to simplify 'handle_Arr_event'. 
    
    Variable:
        args[0] is the arrived service. It is needed to iterate over its functions.
    """
    ev = (clock, handle_manage_FE, args[0])
    insert(ev)

def handle_manage_FE (clock, *args):
    
    """ 
    This event is scheduled at the arrival of a service. So, clock = service arrival time. 
    
    Variable:
        args[0] is the arrived service. It is needed to iterate over its functions.
    """

    # Scheduling loop
    for function in args[0].requested_functions:
        
        """
        Schedule 'Gen_function_event' event for each function in the service based on start times 
            of functions. Pass 'function' and 'function.hosting_node' to the events.
        """
        Gen_function_event(clock, function, function.hosting_node)
            
def Gen_function_event(clock, *args):
    """
    Variable:
        args[0] is a function instance. It is needed to get its completion time, and it is needed 
            in 'Gen_free_resources_event'.
        args[1] is the node hosting the function. It is needed in 'Gen_free_resources_event'.
    """
    
    ev = (clock, handle_function_event, args[0], args[1])
    insert(ev)
    
def handle_function_event(clock, *args):
    """
    Variable:
        args[0] is a function instance. It is needed to get its completion time, and it is needed 
            in 'Gen_free_resources_event'.
        args[1] is the node hosting the function. It is needed in 'Gen_free_resources_event'.
    """
    
    # Generate 'Gen_free_resources_event' based on completion times of functions.
    Gen_free_resources_event(clock, args[0], args[1])

def Gen_free_resources_event(clock, *args):
    """
    Variable:
        args[0] is a function instance. It is needed to get its buffer demand.
        args[1] is the node hosting the function. It is needed to free its resources.
    """
    
    ev = (clock, handle_free_resources_event, args[0], args[1])
    insert(ev)
        
def handle_free_resources_event(clock, *args):
    """
    Variable:
        args[0] is a function instance. It is needed to get its buffer demand.
        args[1] is the node hosting the function. It is needed to free its resources.
    """
    
    args[1].free_resources(args[0].buffer_demand)
    
    # Schedule 'Gen_Dep_event' if the current function is the last function in a service.
    if(args[0].last_function):
        Gen_Dep_event(clock)
            
def Gen_reject_event(clock):
    
    ev = (clock, handle_reject_event)
    insert(ev)
    
def handle_reject_event(clock):
    global rejected
    
    rejected = rejected + 1
    
def Gen_Dep_event(clock):
    
    ev = (clock, handle_Dep_event)
    insert(ev)
    
def handle_Dep_event(clock):
    global departures, active_services
    
    departures = departures + 1
    active_services = active_services - 1

    
def insert(ev):
    global evList
    
    evList.append(ev)
    evList.sort(key=lambda ev: ev[0])

# Parameters

This section provides initializations of parameters and lists used in a simulation.

In [30]:
# Inter Arrival Time.
lamda = 5.0

# Clock and event list.
clock = 0.0
evList = []

# Number of packets (services) to  be simulated
N = 1500

# Count of simulated arrivals, departures, rejected services, and services currently in the system.
arrivals = 0
departures = 0
rejected = 0
active_services = 0


# Servoce ID. It is increminted in every service arrival.
service_ID = 0

# The number of nodes in the network.
number_of_nodes = 50

# The number of functions.
number_of_functions = 10

# The list 'nodes' holds objects of type 'Node' which represent nodes in the network.
nodes = []

# The list 'function_IDs' holds IDs of functions. IDs are from 1 to 'number_of_functions'
function_IDs = [x for x in range(1, number_of_functions + 1)]

"""
'B_binary' is a binaty matrix used to check if function m can be processed by node n. It is indexed by functions row wise and 
    nodes column wise. Row 0 and column 0 are ignored. We consider rows from 1 to 'number_of_functions' and 
    columns from 1 to 'number_of_nodes'. If entry at row m and column n is 1, then function m can be processed by node n. 
    This matrix is represented by beta symbol in the paper.
"""
B_binary = [[0 for x in range(number_of_nodes + 1)] for y in range(number_of_functions + 1)] 


"""
'processing_times' is a binaty matrix used to get processing time of function m on node n. It is indexed by functions row 
    wise and nodes column wise. Row 0 and column 0 are ignored. We consider rows from 1 to 'number_of_functions' and 
    columns from 1 to 'number_of_nodes'. This matrix is represented by rho symbol in the paper.
"""
processing_times = [[0 for x in range(number_of_nodes + 1)] for y in range(number_of_functions + 1)] 

# Output data lists

Acceptance_Ratio = []
Time_Gaps = []
Flow_Time = []
Queue_Length = []
Total_Cost = []
Total_Revenue = []

# Constants to compute Total Cost. They are set by the original paper.
alpha = 0.2
theta = 0.2

# A list to hold arrival times of services to plot performance metrics vs IAT of services.
arrival_times = []

# Simulation

This section provides a single simulation run.

In [31]:
# Reset global variables
reset_global()

# Reseed
random.seed()
    
# Generate node.
generate_nodes()

# First arrival event.
Gen_Arr_event(clock)

# To measure the time
start_time = time()

while (len(evList) != 0):

    ev = evList.pop(0)

    clock = ev[0]

    # To check if the event has 4 arguments (e.g., event for 'Gen_free_resources_event()')
    if(len(ev) == 4):
        ev[1](clock, ev[2], ev[3]) 
    
    # To check if the event has 3 arguments (e.g., event for 'Gen_manage_FE()')
    elif(len(ev) == 3):
        ev[1](clock, ev[2])
        
    # The event has 2 arguments (e.g., event for 'Gen_Arr_event()')
    else:
        ev[1](clock)

    print("Arrivals = ", arrivals)
    print("Departures = ", departures)
    print("rejected = ", rejected)
    print("number of current services", active_services)
    print()

end_time = time()

elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)

print("Simulation is done. It required {} minutes and {} seconds".format(minutes, round(seconds, 4)))

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected =  0
number of current services 1

Arrivals =  1
Departures =  0
rejected

number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  7
rejected =  0
number of current services 1

Arrivals =  8
Departures =  8
rejected =  0
number of current services 0

Arrivals =  9
Departures =  8
rejected =  0
number of current services 1

Arrivals =  9
Departures =  8
rejected =  0
number of current services 1

Arrivals

number of current services 0

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of current services 1

Arrivals =  16
Departures =  15
rejected =  0
number of cu

number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of current services 1

Arrivals =  25
Departures =  24
rejected =  0
number of cu

Departures =  30
rejected =  0
number of current services 1

Arrivals =  31
Departures =  30
rejected =  0
number of current services 1

Arrivals =  31
Departures =  31
rejected =  0
number of current services 0

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =  31
rejected =  0
number of current services 1

Arrivals =  32
Departures =

...

Arrivals =  1472
Departures =  1472
rejected =  0
number of current services 0

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
rejected =  0
number of current services 1

Arrivals =  1473
Departures =  1472
reje

rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1478
rejected =  0
number of current services 1

Arrivals =  1479
Departures =  1479
rejected =  0
number of current services 0

Arrivals =  1480
Departures =  1479
rejected =  0
number of current services 1

Arrivals =  1480
Departures =  1479
rejected =  0
number of current services

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1484
rejected =  0
number of current services 1

Arrivals =  1485
Departures =  1485
rejected =  0
number of current services 0

Arrivals =  1486
Departures =  1485
rejected =  0
number of current services 1

Arrivals =  1486
Departures =  1485
rejected =  0
number of current services 1

Arrivals =  1486
Departures =  1485
rejected =  0
number of current services 1

Arrivals =  1486
Departures =  1485
reje

number of current services 1

Arrivals =  1491
Departures =  1490
rejected =  0
number of current services 1

Arrivals =  1491
Departures =  1490
rejected =  0
number of current services 1

Arrivals =  1491
Departures =  1490
rejected =  0
number of current services 1

Arrivals =  1491
Departures =  1490
rejected =  0
number of current services 1

Arrivals =  1491
Departures =  1491
rejected =  0
number of current services 0

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =  1492
Departures =  1491
rejected =  0
number of current services 1

Arrivals =

Departures =  1496
rejected =  0
number of current services 1

Arrivals =  1497
Departures =  1496
rejected =  0
number of current services 1

Arrivals =  1497
Departures =  1496
rejected =  0
number of current services 1

Arrivals =  1497
Departures =  1497
rejected =  0
number of current services 0

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number of current services 1

Arrivals =  1498
Departures =  1497
rejected =  0
number 